Bài 1

In [ ]:
import cv2
import numpy as np
import os

# Đường dẫn đến thư mục chứa ảnh
folder_path = 'exercise'

# Tạo thư mục lưu ảnh kết quả
output_path = 'output'
os.makedirs(output_path, exist_ok=True)

def load_images():
    images = []
    filenames = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_GRAYSCALE)
            images.append(img)
            filenames.append(file)
    return images, filenames

def image_inverse(img):
    return 255 - img

def gamma_correction(img, gamma=2.2):
    norm_img = img / 255.0
    corrected = np.power(norm_img, gamma)
    return np.uint8(corrected * 255)

def log_transform(img):
    c = 255 / np.log(1 + np.max(img))
    log_image = c * np.log(1 + img.astype(np.float32))
    return np.uint8(log_image)

def histogram_equalization(img):
    return cv2.equalizeHist(img)

def contrast_stretching(img):
    a, b = 0, 255
    min_val, max_val = np.min(img), np.max(img)
    stretched = (img - min_val) * ((b - a) / (max_val - min_val)) + a
    return np.uint8(stretched)

def process_images(method_key):
    images, filenames = load_images()
    for img, name in zip(images, filenames):
        if method_key == 'I':
            result = image_inverse(img)
        elif method_key == 'G':
            result = gamma_correction(img)
        elif method_key == 'L':
            result = log_transform(img)
        elif method_key == 'H':
            result = histogram_equalization(img)
        elif method_key == 'C':
            result = contrast_stretching(img)
        else:
            print("Phím không hợp lệ.")
            return
        
        output_file = os.path.join(output_path, f'{method_key}_{name}')
        cv2.imwrite(output_file, result)
        cv2.imshow(f'{method_key}_{name}', result)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Menu chính
print("Nhấn phím để chọn phương pháp:")
print("I - Image Inverse")
print("G - Gamma Correction")
print("L - Log Transformation")
print("H - Histogram Equalization")
print("C - Contrast Stretching")

key = input("Chọn phương pháp (I/G/L/H/C): ").upper()
process_images(key)


Nhấn phím để chọn phương pháp:
I - Image Inverse
G - Gamma Correction
L - Log Transformation
H - Histogram Equalization
C - Contrast Stretching


Bài 2

In [ ]:
import cv2
import numpy as np
import os

folder_path = 'exercise'
output_path = 'output_fft'
os.makedirs(output_path, exist_ok=True)

def load_images():
    images = []
    filenames = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_GRAYSCALE)
            images.append(img)
            filenames.append(file)
    return images, filenames

def fast_fourier_transform(img):
    dft = np.fft.fft2(img)
    dft_shift = np.fft.fftshift(dft)
    magnitude_spectrum = 20 * np.log(np.abs(dft_shift) + 1)
    magnitude_spectrum = np.uint8(cv2.normalize(magnitude_spectrum, None, 0, 255, cv2.NORM_MINMAX))
    return magnitude_spectrum

def butterworth_lowpass(img, D0=30, n=2):
    rows, cols = img.shape
    crow, ccol = rows // 2 , cols // 2
    dft = np.fft.fft2(img)
    dft_shift = np.fft.fftshift(dft)

    mask = np.zeros_like(img, dtype=np.float32)
    for u in range(rows):
        for v in range(cols):
            D = np.sqrt((u - crow)**2 + (v - ccol)**2)
            mask[u, v] = 1 / (1 + (D / D0)**(2 * n))

    filtered = dft_shift * mask
    f_ishift = np.fft.ifftshift(filtered)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)
    return np.uint8(cv2.normalize(img_back, None, 0, 255, cv2.NORM_MINMAX))

def butterworth_highpass(img, D0=30, n=2):
    rows, cols = img.shape
    crow, ccol = rows // 2 , cols // 2
    dft = np.fft.fft2(img)
    dft_shift = np.fft.fftshift(dft)

    mask = np.zeros_like(img, dtype=np.float32)
    for u in range(rows):
        for v in range(cols):
            D = np.sqrt((u - crow)**2 + (v - ccol)**2)
            if D == 0:
                mask[u, v] = 0
            else:
                mask[u, v] = 1 / (1 + (D0 / D)**(2 * n))

    filtered = dft_shift * mask
    f_ishift = np.fft.ifftshift(filtered)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)
    return np.uint8(cv2.normalize(img_back, None, 0, 255, cv2.NORM_MINMAX))

def process_images(method_key):
    images, filenames = load_images()
    for img, name in zip(images, filenames):
        if method_key == 'F':
            result = fast_fourier_transform(img)
        elif method_key == 'L':
            result = butterworth_lowpass(img)
        elif method_key == 'H':
            result = butterworth_highpass(img)
        else:
            print("Phím không hợp lệ.")
            return
        
        output_file = os.path.join(output_path, f'{method_key}_{name}')
        cv2.imwrite(output_file, result)
        cv2.imshow(f'{method_key}_{name}', result)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Menu
print("Nhấn phím để chọn phương pháp:")
print("F - Fast Fourier")
print("L - Butterworth Lowpass Filter")
print("H - Butterworth Highpass Filter")

key = input("Chọn phương pháp (F/L/H): ").upper()
process_images(key)


Nhấn phím để chọn phương pháp:
F - Fast Fourier
L - Butterworth Lowpass Filter
H - Butterworth Highpass Filter


Bài  3

In [ ]:
import cv2
import numpy as np
import os
import random

# Thư mục ảnh gốc và ảnh đầu ra
folder_path = 'exercise'
output_path = 'output_rgb_random'
os.makedirs(output_path, exist_ok=True)

# Các hàm biến đổi từ Câu 1
def image_inverse(img):
    return 255 - img

def gamma_correction(img, gamma=2.2):
    norm_img = img / 255.0
    corrected = np.power(norm_img, gamma)
    return np.uint8(corrected * 255)

def log_transform(img):
    c = 255 / np.log(1 + np.max(img))
    log_image = c * np.log(1 + img.astype(np.float32))
    return np.uint8(log_image)

def histogram_equalization(img):
    if len(img.shape) == 2:
        return cv2.equalizeHist(img)
    else:
        # Chuyển sang YCrCb và equalize kênh Y
        ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

def contrast_stretching(img):
    a, b = 0, 255
    min_val, max_val = np.min(img), np.max(img)
    stretched = (img - min_val) * ((b - a) / (max_val - min_val)) + a
    return np.uint8(stretched)

# Danh sách các hàm biến đổi
transformations = [image_inverse, gamma_correction, log_transform, histogram_equalization, contrast_stretching]

# ✅ Hoán đổi ngẫu nhiên thứ tự kênh màu RGB (đã sửa lỗi tuple)
def shuffle_rgb_channels(img):
    channels = list(cv2.split(img))  # chuyển tuple -> list để shuffle được
    random.shuffle(channels)
    return cv2.merge(channels)

# Xử lý ảnh
def process_images():
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)

            # Đổi thứ tự kênh màu RGB ngẫu nhiên
            shuffled_img = shuffle_rgb_channels(img)

            # Chuyển ảnh sang xám nếu phép biến đổi yêu cầu (trừ histogram có thể làm màu)
            gray_img = cv2.cvtColor(shuffled_img, cv2.COLOR_BGR2GRAY)

            # Chọn ngẫu nhiên một phép biến đổi từ câu 1
            transformation = random.choice(transformations)

            try:
                # Nếu hàm áp dụng được cho ảnh màu thì dùng màu, không thì dùng xám
                try_img = transformation(shuffled_img)
            except:
                try_img = transformation(gray_img)

            output_file = os.path.join(output_path, f'rgb_{file}')
            cv2.imwrite(output_file, try_img)
            cv2.imshow(f'RGB Transformed - {file}', try_img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

process_images()


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13044\274274046.py:21: RuntimeWarning: overflow encountered in scalar add
  c = 255 / np.log(1 + np.max(img))
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13044\274274046.py:21: RuntimeWarning: divide by zero encountered in log
  c = 255 / np.log(1 + np.max(img))


Bài 4

In [ ]:
import cv2
import numpy as np
import os
import random
from scipy.fft import fft2, ifft2, fftshift, ifftshift
from scipy.ndimage import minimum_filter, maximum_filter

# Thư mục ảnh gốc và đầu ra
folder_path = 'exercise'
output_path = 'output_rgb_butterworth'
os.makedirs(output_path, exist_ok=True)

# ================== Hàm biến đổi ảnh ==================

def shuffle_rgb_channels(img):
    channels = list(cv2.split(img))
    random.shuffle(channels)
    return cv2.merge(channels)

def butterworth_lowpass_filter(img, cutoff=30, order=2):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dft = fftshift(fft2(gray))
    rows, cols = gray.shape
    crow, ccol = rows // 2, cols // 2

    u = np.arange(rows)
    v = np.arange(cols)
    U, V = np.meshgrid(u - crow, v - ccol, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    H = 1 / (1 + (D / cutoff)**(2 * order))

    filtered = dft * H
    img_back = np.abs(ifft2(ifftshift(filtered)))
    return np.uint8(img_back)

def butterworth_highpass_filter(img, cutoff=30, order=2):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dft = fftshift(fft2(gray))
    rows, cols = gray.shape
    crow, ccol = rows // 2, cols // 2

    u = np.arange(rows)
    v = np.arange(cols)
    U, V = np.meshgrid(u - crow, v - ccol, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    H = 1 / (1 + (cutoff / D)**(2 * order))
    H[D == 0] = 0  # tránh chia cho 0

    filtered = dft * H
    img_back = np.abs(ifft2(ifftshift(filtered)))
    return np.uint8(img_back)

def min_filter(img, size=3):
    return minimum_filter(img, size=size)

def max_filter(img, size=3):
    return maximum_filter(img, size=size)

# ================== Xử lý ảnh ==================

def process_images():
    filters = ['lowpass', 'highpass']
    
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)

            # Đổi thứ tự RGB
            shuffled_img = shuffle_rgb_channels(img)

            # Ngẫu nhiên chọn loại filter
            selected = random.choice(filters)

            if selected == 'lowpass':
                print(f"{file}: Butterworth Lowpass + Min Filter")
                result = butterworth_lowpass_filter(shuffled_img)
                result = min_filter(result)
            else:
                print(f"{file}: Butterworth Highpass + Max Filter")
                result = butterworth_highpass_filter(shuffled_img)
                result = max_filter(result)

            output_file = os.path.join(output_path, f'filtered_{file}')
            cv2.imwrite(output_file, result)
            cv2.imshow(f'Transformed - {file}', result)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

process_images()


ha-long-bay-in-vietnam.jpg: Butterworth Highpass + Max Filter


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12668\2939733311.py:46: RuntimeWarning: divide by zero encountered in divide
  H = 1 / (1 + (cutoff / D)**(2 * order))


pagoda.jpg: Butterworth Highpass + Max Filter
quang_ninh.jpg: Butterworth Highpass + Max Filter
